In [1]:
import datetime
import gradio as gr
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


import warnings
warnings.filterwarnings('ignore')

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from langchain.memory import MemoryViewMemory
load_dotenv()


True

In [3]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo-0301


In [4]:
# def chatWithNCAIR(question, history):
load_dotenv()

persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
llm = ChatOpenAI(model_name=llm_name, temperature=0)



template = """If the user says any greetings and says their name like "hello I'm bishop", 
 always reply by greeting the person and saying his name  
 Use the following pieces of context to answer the question at the end. 
 If you don't know the answer, just say that you don't know, don't try to make up an answer. 
 Use only 3 sentences but Keep the answer as concise as possible.
 {context}
 Previous conversation:
 {chat_history}
 New human question: {question}
 Response:"""

# QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question","chat_history"], template=template)
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


print(f"{vectordb._collection.count()} results for vectordb")
print(QA_CHAIN_PROMPT)

# Run chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"
        )
        })
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
    )

retriever=vectordb.as_retriever()


    

353 results for vectordb
input_variables=['chat_history', 'context', 'question'] output_parser=None partial_variables={} template='If the user says any greetings and says their name like "hello I\'m bishop", \n always reply by greeting the person and saying his name  \n Use the following pieces of context to answer the question at the end. \n If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer. \n Use only 3 sentences but Keep the answer as concise as possible.\n {context}\n Previous conversation:\n {chat_history}\n New human question: {question}\n Response:' template_format='f-string' validate_template=True


In [5]:
def chatbot(prompt, history):   
    qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    get_chat_history=lambda h : h,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT}
    )


    result = qa({"question": prompt})
    return result["answer"]

In [6]:
# persist_directory = 'docs/chroma/'
# embedding = OpenAIEmbeddings()
# vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
# vectordb._collection.count()

In [7]:
chatbot("Who is the director of NCAIR?","")

'The director of NCAIR is Mr Kashif Inuwa Abdullahi CCIE, as stated in the signed message by DG/CEO NITDA.'

In [8]:
chatbot("Tell about him?","")

'Mallam Kashifu Inuwa Abdullahi CCIE is the Director General and Chief Executive Officer of the National Information Technology Development Agency (NITDA) and also the director of NCAIR. He is a thoroughbred IT professional with an impressive resume in public and private sector IT development, policy formulation, and IT governance. He holds various executive certificates from top-notch universities such as Harvard University, USA, University of Cambridge and Oxford University, UK. His strategic prowess is anchored in training from the world-renowned Massachusetts Institute of Technology (MIT).'

In [11]:
demo = gr.ChatInterface(fn=chatbot,
    chatbot=gr.Chatbot(height=300, min_width=40),
    # textbox=gr.Textbox(placeholder="Ask me a question relating to NCAIR"),
    title="Chat with NCAIR💬",
    description="Ask NCAIR any question",
    theme="soft",
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
    )

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://24b9f554a270bb27e1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-K0yZI60wc0q5sQ7XTUrjhAAC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-K0yZI60wc0q5sQ7XTUrjhAAC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht